### 帥哥 Load Data:

- Get the historical data of bitcoin, eth and ada in csv.
- get the volume, open price, close price, high price and low price with the interval 15min, 1h, 4h, 1d, 1w (separate csv)

In [2]:
!pip install pandas
!pip install requests


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 889.7 kB/s eta 0:00:00a 0:00:01
Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl (196 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
from pathlib import Path

def get_binance_data(symbol, interval, start_time, end_time):
    """
    Fetch historical kline data from Binance
    
    Additional fields we'll include:
    - Number of trades
    - Quote asset volume (Volume in USDT)
    - Taker buy volume
    - Taker buy quote volume
    """
    
    endpoint = "https://api.binance.com/api/v3/klines"
    
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_time.timestamp() * 1000),
        "endTime": int(end_time.timestamp() * 1000),
        "limit": 1000
    }
    
    data = []
    while start_time < end_time:
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            batch = response.json()
            if not batch:
                break
                
            data.extend(batch)
            
            # Update start time for next batch
            start_time = datetime.fromtimestamp(batch[-1][0]/1000) + timedelta(minutes=1)
            params["startTime"] = int(start_time.timestamp() * 1000)
            
            # Respect rate limits
            time.sleep(0.1)
        else:
            print(f"Error: {response.status_code}")
            break
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'
    ])
    
    # Clean up the data
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.drop(['close_time', 'ignore'], axis=1)
    
    # Convert string values to float
    numeric_columns = ['open', 'high', 'low', 'close', 'volume', 
                      'quote_asset_volume', 'taker_buy_volume', 
                      'taker_buy_quote_volume']
    df[numeric_columns] = df[numeric_columns].astype(float)
    
    return df

# Define parameters
symbols = ['BTCUSDT', 'ADAUSDT', 'ETHUSDT']
intervals = {
    '15m': '15m',
    '1h': '1h',
    '4h': '4h',
    '1d': '1d',
    '1w': '1w'
}

# Calculate time range (5 years ago from now)
end_time = datetime.now()
start_time = end_time - timedelta(days=5*365)

# Create output directory
output_dir = Path('crypto_data')
output_dir.mkdir(exist_ok=True)

# Fetch data for each symbol and interval
for symbol in symbols:
    print(f"\nFetching data for {symbol}")
    
    for interval_name, interval_code in intervals.items():
        print(f"Processing {interval_name} interval...")
        
        df = get_binance_data(symbol, interval_code, start_time, end_time)
        
        # Create filename
        filename = output_dir / f"{symbol}_{interval_name}.csv"
        
        # Save to CSV
        df.to_csv(filename, index=False)
        print(f"Saved to {filename}")



Fetching data for BTCUSDT
Processing 15m interval...
Saved to crypto_data/BTCUSDT_15m.csv
Processing 1h interval...
Saved to crypto_data/BTCUSDT_1h.csv
Processing 4h interval...
Saved to crypto_data/BTCUSDT_4h.csv
Processing 1d interval...
Saved to crypto_data/BTCUSDT_1d.csv
Processing 1w interval...
Saved to crypto_data/BTCUSDT_1w.csv

Fetching data for ADAUSDT
Processing 15m interval...
Saved to crypto_data/ADAUSDT_15m.csv
Processing 1h interval...
Saved to crypto_data/ADAUSDT_1h.csv
Processing 4h interval...
Saved to crypto_data/ADAUSDT_4h.csv
Processing 1d interval...
Saved to crypto_data/ADAUSDT_1d.csv
Processing 1w interval...
Saved to crypto_data/ADAUSDT_1w.csv

Fetching data for ETHUSDT
Processing 15m interval...
Saved to crypto_data/ETHUSDT_15m.csv
Processing 1h interval...
Saved to crypto_data/ETHUSDT_1h.csv
Processing 4h interval...
Saved to crypto_data/ETHUSDT_4h.csv
Processing 1d interval...
Saved to crypto_data/ETHUSDT_1d.csv
Processing 1w interval...
Saved to crypto_dat

### Vincent Load Data: